# Pour P., lister les limites de qualités dans la data
***
_Les parties indispensables à run sont indiquées avec la mention [TO RUN], afin de reproduire les analyses se situant sous ces sections._

**Contexte** :<br>
Il y aurait 467 paramètres sans limite de qualité dans nos données qui ont été identifiés<br>
On veut fournir à Pauline cette liste


**Choses à faire** :
- Sur les données 2024, recroiser avec les catégories de Pauline, et ajouter une colonne à son fichier avec la (ou les) limites trouvées (distinctes).
- Si pas de limite trouvée, laisser vide.
- Notebook qui exporte un fichier Excel


# Résumé de l'étude
***

**1. Exploration**<br>
Je trouve 540 paramètres sans limite de qualité ( `limitequal IS NULL` ) dans la table `edc_resultats` complète (sans filtre sur l'année)<br>
et 403 paramètres sans limite de qualité en 2024 uniquement.<br>
==> je choisi de rester sur l'intégralité des données pour avoir le plus de données possible

**2. Livrable**<br>
Ajout des données de limite de qualité dans la table de mapping de Pauline : <br>
J'importe la liste complète dans un fichier excel (**D4G_limite_de_qualite.xlsx**) <br>

Dans ce fichier, il y a :
- 838 lignes sans valeurs pour limitequal
- 773 lignes avec une seule valeur pour limitequal
- 1 ligne avec 2 valeurs pour limitequal : il s'agit de 1339/NO2

_Note_ : CYANO53 est deux fois dans la table de mapping ( une fois avec `libminparametre` vide et une aure fois remplie)

# Etude
***
### Packages + read data

In [1]:
# [TO RUN]  Packages
import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

In [2]:
# [TO RUN]
import duckdb
from pipelines.tasks._common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [3]:
# pour pd.to_excel
!uv pip install openpyxl 

Using Python 3.12.9 environment at: C:\Users\Vinca\Documents\D4G_2025\13_pollution_eau\.venv
Audited 1 package in 18ms


###  Exploration

In [4]:
# [OPTIONAL] Tables dispo dans la bdd
con.sql("SHOW TABLES").show()

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ edc_communes            │
│ edc_prelevements        │
│ edc_resultats           │
│ mapping_categories      │
│ stg_edc__communes       │
│ stg_edc__prevelevements │
│ stg_edc__resultats      │
└─────────────────────────┘



In [5]:
# [OPTIONAL] Preview edc_resultats
preview_resultats = con.sql("SELECT * FROM edc_resultats LIMIT 2").df()
preview_resultats

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
0,001,00100119085,12DCLE,1161,"DICHLOROÉTHANE-1,2","Dichloroéthane-1,2",None,N,L,"<0,50",µg/L,133,<=3 µg/L,None,0.0,107-06-2,00100125759,2020,2025-02-14,20230811-150005
1,001,00100119085,A2H,1832,ATRAZINE-2-HYDROXY,Atrazine-2-hydroxy,None,N,L,"<0,020",µg/L,133,"<=0,1 µg/L",None,0.0,2163-68-0,00100125759,2020,2025-02-14,20230811-150005


In [6]:
# [OPTIONAL] Preview mapping_categories
preview_mapping_categories = con.sql("SELECT * FROM mapping_categories LIMIT 2").df()
preview_mapping_categories

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,PESTOT,6276,TOTAL DES PESTICIDES ANALYSÉS,Total des pesticides analysés,None,pesticides
1,ATRZ,1107,ATRAZINE,Atrazine,1912-24-9,pesticides


In [7]:
con.sql("SELECT COUNT (*) FROM mapping_categories").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1612 │
└──────────────┘



### Identification des paramètre sans limite de qualité

In [8]:
con.sql(
    "SELECT COUNT(DISTINCT cdparametre ) FROM   edc_resultats WHERE limitequal IS NULL"
).show()

┌─────────────────────────────┐
│ count(DISTINCT cdparametre) │
│            int64            │
├─────────────────────────────┤
│                         540 │
└─────────────────────────────┘



In [9]:
con.sql(
    "SELECT COUNT(DISTINCT cdparametre ) FROM   edc_resultats WHERE limitequal IS NULL AND de_partition='2024'"
).show()

┌─────────────────────────────┐
│ count(DISTINCT cdparametre) │
│            int64            │
├─────────────────────────────┤
│                         403 │
└─────────────────────────────┘



In [10]:
query_no_param_limitequal = """ 
SELECT 
  cdparametre,
  STRING_AGG(DISTINCT libmajparametre) AS list_libmajparametre  ,
  STRING_AGG(DISTINCT cdparametresiseeaux) AS list_cdparametresiseeaux,
  STRING_AGG(DISTINCT casparam) AS list_casparam,
FROM  
    edc_resultats 
WHERE
    limitequal IS NULL 
    AND cdparametre IS NOT NULL
GROUP BY 
    cdparametre
"""

df_no_param_limitequal = con.sql(query_no_param_limitequal).df()
df_no_param_limitequal

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cdparametre,list_libmajparametre,list_cdparametresiseeaux,list_casparam
0,1384,"VANADIUM,VANADIUM DISSOUS","VA,VAD",7440-62-2
1,1292,XYLÈNE ORTHO,XYLO,95-47-6
2,5901,ODEUR (QUALITATIF),ODQ,None
3,1338,SULFATES,SO4,None
4,1165,"DICHLOROBENZÈNE-1,2",12DCB,95-50-1
...,...,...,...,...
535,1629,"TRICHLORO-1,3,5-BENZÈNE",135TCB,108-70-3
536,1328,CARBONATES,CO3,None
537,1356,TOXICITÉ DAPHNIES 24H,TOXD24,None
538,7742,CEL. DE CYANOBACTÉRIES TOXINOGÈNES,CYANTOX,None


### Identification des limites de qualité dans les données

In [11]:
# [TO RUN]

query_check_limitequal = """ 
WITH CAT AS (
    SELECT
      *
    FROM
      mapping_categories
),
LIM AS (
SELECT DISTINCT
  cdparametre,
  libmajparametre,
  cdparametresiseeaux,
  casparam,
  limitequal,
  CAST(regexp_extract(REPLACE("limitequal", ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
  regexp_extract("limitequal", '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite
FROM  
    edc_resultats 
)
SELECT 
  CAT.*,
  LIM.limitequal,
  LIM.limitequal_float,
  LIM.unite
FROM  
    CAT 
LEFT JOIN
    LIM
ON 
  LIM.cdparametre =   CAT.cdparametre
  AND LIM.libmajparametre = CAT.libmajparametre
  AND  LIM.cdparametresiseeaux = CAT.cdparametresiseeaux
  AND  LIM.casparam = CAT.casparam
"""

df_check_limitequal = con.sql(query_check_limitequal).df()
df_check_limitequal

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_660\3641305764.py:3: SyntaxWarning: invalid escape sequence '\d'
  query_check_limitequal = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,unite
0,ALTMICR,1370,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,7429-90-5,minéral,None,NaN,None
1,AS,1369,ARSENIC,Arsenic,7440-38-2,métaux lourds,<=10 µg/L,10.0,µg/L
2,BRF,1122,BROMOFORME,Bromoforme,75-25-2,sous produit désinfection,<=100 µg/L,100.0,µg/L
3,CA,1374,CALCIUM,Calcium,7440-70-2,minéral,None,NaN,None
4,CYROMAZ,2897,CYROMAZINE,Cyromazine,66215-27-8,pesticides,"<=0,1 µg/L",0.1,µg/L
...,...,...,...,...,...,...,...,...,...
1611,RBETA2R,2955,ACTIVITÉ BÉTA GLOB. RÉSIDUELLE BQ/L,Activité béta glob. résiduelle Bq/L,None,radioactivité,None,NaN,None
1612,TACE,<NA>,TAC À L'ÉQUILIBRE,TAC à l'équilibre,None,non classé,None,NaN,None
1613,TRZ_TOT,1282,TRIAZINES,Triazines,None,non classé,None,NaN,None
1614,NEB,1425,NÉBULOSITÉ,Nébulosité,None,non classé,None,NaN,None


On passe de 1612 à 1616 , il y a quelques subtances avec plusieurs valeurs de limitequal

In [12]:
duplicate = df_check_limitequal[
    df_check_limitequal.duplicated(
        ["cdparametre", "libmajparametre", "cdparametresiseeaux", "casparam"]
    )
]
duplicate

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,unite
487,BMG,1362,BORE MG/L,Bore mg/L,7440-42-8,non classé,None,NaN,None
886,SE,1385,SÉLÉNIUM,Sélénium,7782-49-2,métaux lourds,<=20 µg/L,20.0,µg/L
1054,NORFLDM,2737,DESMETHYLNORFLURAZON,Desmethylnorflurazon,23576-24-1,métabolite de pesticide,None,NaN,None
1084,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite,"<=0,1 mg/L",0.1,mg/L
1393,CYANO53,<NA>,JAAGINEMA SP,Jaaginema sp,None,microbio,None,NaN,None


In [13]:
list_of_values = duplicate["cdparametresiseeaux"]

df_check_limitequal[
    df_check_limitequal["cdparametresiseeaux"].isin(list_of_values)
].sort_values(by=["cdparametre"])

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,unite
806,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite,"<=0,5 mg/L",0.5,mg/L
1084,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite,"<=0,1 mg/L",0.1,mg/L
1580,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),None,nitrite,None,NaN,None
299,BMG,1362,BORE MG/L,Bore mg/L,7440-42-8,non classé,"<=1,5 mg/L",1.5,mg/L
487,BMG,1362,BORE MG/L,Bore mg/L,7440-42-8,non classé,None,NaN,None
786,SE,1385,SÉLÉNIUM,Sélénium,7782-49-2,métaux lourds,None,NaN,None
886,SE,1385,SÉLÉNIUM,Sélénium,7782-49-2,métaux lourds,<=20 µg/L,20.0,µg/L
666,NORFLDM,2737,DESMETHYLNORFLURAZON,Desmethylnorflurazon,23576-24-1,métabolite de pesticide,"<=0,1 µg/L",0.1,µg/L
1054,NORFLDM,2737,DESMETHYLNORFLURAZON,Desmethylnorflurazon,23576-24-1,métabolite de pesticide,None,NaN,None
1392,CYANO53,<NA>,JAAGINEMA SP,None,None,microbio,None,NaN,None


# Livrable 

In [14]:
# [TO RUN]

query_finale = """ 
WITH CAT AS (
    SELECT
      *
    FROM
      mapping_categories
),
LIM AS (
    SELECT 
      cdparametre,
      libmajparametre,
      cdparametresiseeaux,
      casparam,
      STRING_AGG(DISTINCT limitequal) AS limitequal,
      STRING_AGG(DISTINCT CAST(regexp_extract(REPLACE("limitequal", ',', '.'), '-?\d+(\.\d+)?') AS FLOAT)) AS limitequal_float,
      STRING_AGG(DISTINCT regexp_extract("limitequal", '[a-zA-Zµg]+/?[a-zA-Z/L]+$')) AS limitequal_unite,
      COUNT(DISTINCT limitequal) As nb_limitequal
    FROM  
        edc_resultats 
    GROUP BY 
      cdparametre,
      libmajparametre,
      cdparametresiseeaux,
      casparam
)
SELECT 
  CAT.*,
  LIM.limitequal,
  LIM.limitequal_float,
  LIM.limitequal_unite,
  COALESCE(LIM.nb_limitequal,0) AS nb_limitequal
FROM  
    CAT 
LEFT JOIN
    LIM
ON 
  LIM.cdparametre =   CAT.cdparametre
  AND LIM.libmajparametre = CAT.libmajparametre
  AND  LIM.cdparametresiseeaux = CAT.cdparametresiseeaux
  AND  LIM.casparam = CAT.casparam
"""

df_final = con.sql(query_finale).df()
df_final

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_660\447479714.py:3: SyntaxWarning: invalid escape sequence '\d'
  query_finale = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,limitequal_unite,nb_limitequal
0,ATRZ,1107,ATRAZINE,Atrazine,1912-24-9,pesticides,"<=0,1 µg/L",0.1,µg/L,1
1,SMZ,1263,SIMAZINE,Simazine,122-34-9,pesticides,"<=0,1 µg/L",0.1,µg/L,1
2,MTC,1221,MÉTOLACHLORE,Métolachlore,51218-45-2,pesticides,"<=0,1 µg/L",0.1,µg/L,1
3,TBZ,1268,TERBUTHYLAZIN,Terbuthylazin,5915-41-3,pesticides,"<=0,1 µg/L",0.1,µg/L,1
4,MTBZ,1225,MÉTRIBUZINE,Métribuzine,21087-64-9,pesticides,"<=0,1 µg/L",0.1,µg/L,1
...,...,...,...,...,...,...,...,...,...,...
1607,SMETOLA,2974,S-MÉTOLACHLORE,S-Métolachlore,None,pesticides,None,None,None,0
1608,BRMTH,1502,BIORESMETHRINE,Bioresmethrine,28434-01-07,pesticides,None,None,None,0
1609,PHRT,1525,PHORATE,Phorate,0298-02-02,pesticides,None,None,None,0
1610,DICOUM,2982,DIFENACOUM,Difenacoum,56073-07-05,pesticides,None,None,None,0


In [15]:
df_final[df_final["cdparametresiseeaux"].isin(list_of_values)].sort_values(
    by=["cdparametre"]
)

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,limitequal_unite,nb_limitequal
676,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite,"<=0,5 mg/L,<=0,1 mg/L","0.5,0.1",mg/L,2
1184,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),None,nitrite,None,None,None,0
961,BMG,1362,BORE MG/L,Bore mg/L,7440-42-8,non classé,"<=1,5 mg/L",1.5,mg/L,1
711,SE,1385,SÉLÉNIUM,Sélénium,7782-49-2,métaux lourds,<=20 µg/L,20.0,µg/L,1
593,NORFLDM,2737,DESMETHYLNORFLURAZON,Desmethylnorflurazon,23576-24-1,métabolite de pesticide,"<=0,1 µg/L",0.1,µg/L,1
1385,CYANO53,<NA>,JAAGINEMA SP,None,None,microbio,None,None,None,0
1389,CYANO53,<NA>,JAAGINEMA SP,Jaaginema sp,None,microbio,None,None,None,0


In [16]:
df_final.groupby("nb_limitequal").count()

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,limitequal_unite
nb_limitequal,,,,,,,,,
0,837,777,838,834,440,838,0,0,0
1,773,773,773,773,773,773,773,773,773
2,1,1,1,1,1,1,1,1,1


In [17]:
# To excel
df_final.to_excel("D4G_limite_de_qualite.xlsx")